In [6]:
import pandas as pd
import numpy as np
import math

import os

from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, learning_curve, KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder, StandardScaler, RobustScaler, FunctionTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [7]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [8]:
df = pd.read_csv('data_5k_lignes.csv')

df.head(10)

,artist_name,album_name,album_id,track_name,track_id,popularity,release_date,duration_ms,explicit,external_urls,is_local,preview_url,track_number,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms.1,time_signature
0,Gen Hoshino,Comedy,41ERrwfzos93Xlf6hFBiDn,Comedy,5SuOikwiRyPMVoIQDJUgSV,66,2022-04-08,230666,False,https://open.spotify.com/track/5SuOikwiRyPMVoI...,False,https://p.scdn.co/mp3-preview/56ee6d51af5c3a28...,1,0.676,0.461,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,audio_features,5SuOikwiRyPMVoIQDJUgSV,spotify:track:5SuOikwiRyPMVoIQDJUgSV,https://api.spotify.com/v1/tracks/5SuOikwiRyPM...,https://api.spotify.com/v1/audio-analysis/5SuO...,230667,4
1,Ray LaMontagne,I Was Born To Love You,7cwHkTgTHLbCUFB6hRi9QQ,I Was Born To Love You,3PG6V5yuFfo4APiovOQoRv,64,2022-04-08,265843,False,https://open.spotify.com/track/3PG6V5yuFfo4APi...,False,https://p.scdn.co/mp3-preview/46ff8588929ba7bd...,1,0.650,0.271,9,-11.081,1,0.0314,0.7940,0.000069,0.1960,0.210,78.036,audio_features,3PG6V5yuFfo4APiovOQoRv,spotify:track:3PG6V5yuFfo4APiovOQoRv,https://api.spotify.com/v1/tracks/3PG6V5yuFfo4...,https://api.spotify.com/v1/audio-analysis/3PG6...,265843,4
2,Steve Moakler,Make A Little Room,457Ok9pnF5fAfG7lNUFqHS,Make A Little Room,62yXckrKNy9Sylka6oifst,53,2022-08-12,180493,False,https://open.spotify.com/track/62yXckrKNy9Sylk...,False,https://p.scdn.co/mp3-preview/5dcabbd94b669a08...,1,0.758,0.722,1,-6.252,1,0.0511,0.1650,0.000006,0.1060,0.660,146.031,audio_features,62yXckrKNy9Sylka6oifst,spotify:track:62yXckrKNy9Sylka6oifst,https://api.spotify.com/v1/tracks/62yXckrKNy9S...,https://api.spotify.com/v1/audio-analysis/62yX...,180493,4
3,Ben Rector,The Joy of Music,1l4VD485oDSJM23eWJEWJN,Steady Love,6UOkF086zUgYXnqFxqUvEI,55,2022-03-11,224333,False,https://open.spotify.com/track/6UOkF086zUgYXnq...,False,https://p.scdn.co/mp3-preview/173b473ca21bbc2c...,4,0.678,0.277,11,-10.537,1,0.0602,0.8430,0.000004,0.1040,0.238,76.935,audio_features,6UOkF086zUgYXnqFxqUvEI,spotify:track:6UOkF086zUgYXnqFxqUvEI,https://api.spotify.com/v1/tracks/6UOkF086zUgY...,https://api.spotify.com/v1/audio-analysis/6UOk...,224333,4
4,Ben Rector,The Joy of Music,1l4VD485oDSJM23eWJEWJN,Living My Best Life,0R7EWhquaAICmyE5MZqt3q,54,2022-03-11,215213,False,https://open.spotify.com/track/0R7EWhquaAICmyE...,False,https://p.scdn.co/mp3-preview/697c72a3a0750cd9...,3,0.566,0.733,4,-5.302,1,0.0699,0.0830,0.000002,0.3620,0.636,178.828,audio_features,0R7EWhquaAICmyE5MZqt3q,spotify:track:0R7EWhquaAICmyE5MZqt3q,https://api.spotify.com/v1/tracks/0R7EWhquaAIC...,https://api.spotify.com/v1/audio-analysis/0R7E...,215213,4
5,Drew Holcomb,Coming Home: A Collection of Songs,6fuKDHZ9UfLmssjdRTjPfz,Coming Home,2C3gh14oarzTtv3aY3HJ0m,52,2022-01-14,146360,False,https://open.spotify.com/track/2C3gh14oarzTtv3...,False,https://p.scdn.co/mp3-preview/8cdbcef6f8493144...,1,0.625,0.810,10,-5.995,1,0.0330,0.1460,0.000027,0.1350,0.741,104.066,audio_features,2C3gh14oarzTtv3aY3HJ0m,spotify:track:2C3gh14oarzTtv3aY3HJ0m,https://api.spotify.com/v1/tracks/2C3gh14oarzT...,https://api.spotify.com/v1/audio-analysis/2C3g...,146360,4
6,Penny and Sparrow,Olly Olly,4ruy6zMB9t2sKrNyK2B8zF,Need You,7f0PCIJFVw7U5tbuFH4rb0,54,2022-01-21,189282,False,https://open.spotify.com/track/7f0PCIJFVw7U5tb...,False,https://p.scdn.co/mp3-preview/3cafe5e4ac661f04...,3,0.545,0.642,10,-7.344,1,0.0279,0.5170,0.000002,0.1160,0.555,78.510,audio_features,7f0PCIJFVw7U5tbuFH4rb0,spotify:track:7f0PCIJFVw7U5tbuFH4rb0,https://api.spotify.com/v1/tracks/7f0PCIJFVw7U...,https://api.spotify.com/v1/audio-analysis/7f0P...,189282,4
7,Penny and Sparrow,Olly Olly,4ruy6zMB9t2sKrNyK2B8zF,Alabama Haint,2Z7yllvjIQhTpCa5909jLs,51,2022-01-21,182601,False,https://open.spotify.com/track/2Z7yllvjIQhTpCa...,False,https://p.scdn.co/mp3-preview/6d7b51a77270b008...,2,0.589,0.422,4,-10.780,0,0.0296,0.8680,0.027900,0.0897,0.293,94.968,audio_features,2Z7yllvjIQhTpCa5

In [9]:
df.drop(df.select_dtypes('object').columns, axis=1, inplace=True)
df['explicit'] = df['explicit'].apply(lambda x : 0 if x == False else 1)
df['is_local'] = df['is_local'].apply(lambda x : 0 if x == False else 1)

df

,popularity,duration_ms,explicit,is_local,track_number,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms.1,time_signature
0,66,230666,0,0,1,0.676,0.461,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,230667,4
1,64,265843,0,0,1,0.650,0.271,9,-11.081,1,0.0314,0.7940,0.000069,0.1960,0.2100,78.036,265843,4
2,53,180493,0,0,1,0.758,0.722,1,-6.252,1,0.0511,0.1650,0.000006,0.1060,0.6600,146.031,180493,4
3,55,224333,0,0,4,0.678,0.277,11,-10.537,1,0.0602,0.8430,0.000004,0.1040,0.2380,76.935,224333,4
4,54,215213,0,0,3,0.566,0.733,4,-5.302,1,0.0699,0.0830,0.000002,0.3620,0.6360,178.828,215213,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5645,37,346200,0,0,1,0.464,0.517,5,-6.384,1,0.0303,0.5920,0.000000,0.1080,0.4410,75.120,346200,4
5646,36,262506,0,0,1,0.453,0.706,2,-6.527,1,0.0278,0.1130,0.000000,0.0754,0.2980,80.000,262507,4
5647,40,532876,0,0,6,0.210,0.525,10,-8.114,1,0.0353,0.5250,0.000812,0.1630,0.0661,159.974,532876,4
5648,39,290633,0,0,1,0.538,0.797,10,-6.794,1,0.0568,0.0140,0.000000,0.0966,0.2100,137.036,290633,4


In [10]:
df.dtypes

popularity            int64
duration_ms           int64
explicit              int64
is_local              int64
track_number          int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms.1         int64
time_signature        int64
dtype: object

In [11]:
X = df.drop('popularity', axis=1)
y = df['popularity']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.95, random_state=42)
X_train.head()

,duration_ms,explicit,is_local,track_number,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms.1,time_signature
5143,271312,1,0,8,0.842,0.588,10,-6.681,1,0.0479,0.1560,0.00000,0.3450,0.477,106.986,271313,4
4335,131977,1,0,1,0.876,0.669,8,-4.742,1,0.3730,0.0256,0.00000,0.2270,0.286,143.958,131977,4
4406,199148,1,0,1,0.848,0.673,11,-4.034,0,0.0580,0.0147,0.00652,0.1020,0.390,93.987,199149,4
4644,190013,0,0,12,0.361,0.138,0,-17.839,1,0.0440,0.9750,0.00000,0.0985,0.386,93.794,190013,4
4239,160114,0,0,2,0.547,0.446,2,-5.288,1,0.0307,0.2100,0.00000,0.0929,0.572,130.016,160114,4


In [13]:
dummy = DummyRegressor()

dummy.fit(X_train, y_train)
dummy.score(X_test, y_test)

-0.00017275314720510693

In [14]:
df.columns

Index(['popularity', 'duration_ms', 'explicit', 'is_local', 'track_number',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms.1', 'time_signature'],
      dtype='object')

In [15]:
var = X.columns

transformer = StandardScaler()

encoder = ColumnTransformer(
    transformers=[
        ('var', transformer, var)
    ]
)

In [16]:
models = [Lasso(), Ridge(), ElasticNet(), LinearRegression(), GradientBoostingRegressor()]

In [17]:
list_stacking = []
for model in models:
    list_stacking.append((f"{model.__class__.__name__}", model))

stacking = StackingRegressor(list_stacking, models[0])

models.append(stacking)

list_models = []

for model in models:
    pipe = Pipeline([
        ('enc', encoder), 
        ('poly', PolynomialFeatures(2)), 
        ('model', model),
    ])

    pipe.fit(X_train, y_train)
    score = pipe.score(X_test, y_test)
    score = round(score,4)

    prediction = pipe.predict(X_test)
    score_rmse = math.sqrt(mean_squared_error(y_test, prediction))
    score_mae = mean_absolute_error(y_test, prediction)

    list_models.append([f"{model.__class__.__name__}", f"{round(score *100, 2)}%", f"{round(score_rmse,2)}"])
    
    print(f"Nom du modèle : {model.__class__.__name__}, R2 score : {round(score *100, 2)}%, RMSE score : {round(score_rmse,2)}, MAE : {round(score_mae,2)}")

Nom du modèle : Lasso, R2 score : 26.46%, RMSE score : 15.02, MAE : 11.97
Nom du modèle : Ridge, R2 score : 33.61%, RMSE score : 14.28, MAE : 11.54
Nom du modèle : ElasticNet, R2 score : 26.98%, RMSE score : 14.97, MAE : 11.91
Nom du modèle : LinearRegression, R2 score : 32.79%, RMSE score : 14.36, MAE : 11.59
Nom du modèle : GradientBoostingRegressor, R2 score : 39.82%, RMSE score : 13.59, MAE : 10.86
Nom du modèle : StackingRegressor, R2 score : 40.53%, RMSE score : 13.51, MAE : 10.85


In [18]:
df_list_models = pd.DataFrame(list_models,columns=['modele', "r2_score", "score_rmse"])

df_list_models.sort_values('r2_score', ascending=False)

,modele,r2_score,score_rmse
5,StackingRegressor,40.53%,13.51
4,GradientBoostingRegressor,39.82%,13.59
1,Ridge,33.61%,14.28
3,LinearRegression,32.79%,14.36
2,ElasticNet,26.98%,14.97
0,Lasso,26.46%,15.02


In [19]:
df['popularity'].describe()

count    5650.000000
mean       43.314159
std        17.406561
min         0.000000
25%        33.000000
50%        45.000000
75%        56.000000
max        91.000000
Name: popularity, dtype: float64

# MLFlow

In [21]:
#from run_experiment import run_experiment
import mlflow

In [22]:
experiment_name = "StackingRegressor"
experiment_id = mlflow.create_experiment(experiment_name)

eval_data = X_test
eval_data["label"] = y_test

with mlflow.start_run(experiment_id=experiment_id, nested=True, run_name=experiment_name):
    mlflow.log_params(clf_xgb_with_PCA.named_steps['xgbclassifier'].get_params())
    mlflow.sklearn.log_model(clf_xgb_with_PCA, "model XGB with PCA")
    model_uri = mlflow.get_artifact_uri("model XGB with PCA")

    result = mlflow.evaluate(
            model_uri,
            eval_data,
            targets="label",
            model_type="classifier",
            evaluators=["default"],
    )
    

NameError: name 'clf_xgb_with_PCA' is not defined